# Projeto 4 - Lógica Computacional

Grupo G20

Nome: Lara Catarina Vilaça Lopes ; Número de aluno:a108655

Nome: Fábio Mendes Castelhano ; Número de aluno:a105728

## 1. Abordagem utilizada para resolver o problema

1. Utilização da biblioteca pysmt

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import *

2. Definir as variáveis do navio A

In [2]:

# (0,1) origem no canto inferior esquerdo
# ============================================================
# 1. VARIÁVEIS DO NAVIO A 
# ============================================================
def VAR_A_B(i):
    s = {}
    # estado discreto
    s["sector_A"] = Symbol("sector_A" + str(i), INT)

    # variáveis contínuas de estado
    s["z_A"]   = Symbol("z_A"+ str(i), REAL)
    s["v_A"]   = Symbol("v_A"+ str(i), REAL)
    s["tau_A"] = Symbol("tau_A"+ str(i), REAL)

    #sendo $$(x_0,y_0)\,$$ o ponto do sector onde a trajetória se inicia, no inicio inicio é (0,1)
    s["x0_A"] = Symbol("x0_A"+ str(i), REAL)
    s["y0_A"] = Symbol("y0_A"+ str(i), REAL)

    # parâmetros associados ao modo (sector)
    # Os modos do navio  são definidos pelos sectores $$\,s\,$$. 
    # A cada modo estão associados  parâmetros racionais, constantes dentro de cada sector mas variando de sector para sector.

    s["gamma_A"]   = Symbol("gamma_A"+ str(i), REAL) #  aceleração linear  definida pela propulsão do navio
    s["epsilon_A"] = Symbol("epsilon_A"+ str(i), REAL) #é a  força exercida pela corrente no canal que actua a favor ou contra a trajectória
    s["phi_A"]     = Symbol("phi_A"+ str(i), REAL) #  é o rumo do navio definido como o ângulo que a trajetória do movimento linear faz com o eixo horizontal.
    s["V_A"]       = Symbol("V_A"+ str(i), REAL) # limite superior na velocidade do navio

    # o coeficiente de atrito com a àgua, uma constante positiva que é independente do modo mas depende do navio.
    s["sigma_A"] = Symbol("sigma_A"+ str(i), REAL)

    s["v_A_dot"] = Symbol("v_A_dot", REAL)
    s["z_A_dot"] = Symbol("z_A_dot", REAL)
    return s

3. Definir as zonas 

Zona de aceleração : $$\{s_{11},s_{13}\}\,$$ $$\{s_2,s_4\}\,$$  $$\,\gamma \simeq 1\,$$
Zona de desaceleração : $$\,\{s_{1},s_{3}\}\,$$  $$\,\{s_{12},s_{14}\}\,$$ $$\,\gamma \simeq 0\,$$
Velocidade constante e de cruzeiro : $$\{s_5,s_7,s_9\}\,$$  $$\{s_6,s_8,s_{10}\}\,$$
Zona de velocidade baixa mais ou menos constante : $$\{s_0\}\,$$

In [3]:
# os modos de aceleracao e assim
def acceleration_zone_A_B(s):
    return Or(
        Equals(s["sector_A"], Int(11)),
        Equals(s["sector_A"], Int(13)),
        Equals(s["sector_A"], Int(2)),
        Equals(s["sector_A"], Int(4)),
    )

def deceleration_zone_A_B(s):
    return Or(
        Equals(s["sector_A"], Int(1)),
        Equals(s["sector_A"], Int(3)),
        Equals(s["sector_A"], Int(12)),
        Equals(s["sector_A"], Int(14)),
    )
def v_constant(sector):
    return Or(
        Equals(sector, Int(5)), Equals(sector, Int(7)), Equals(sector, Int(9)),
        Equals(sector, Int(6)), Equals(sector, Int(8)), Equals(sector, Int(10))
    )

def low_speed(sector):
    return Equals(sector, Int(0))

def mode_A_B(s):
    return Or(
        acceleration_zone_A_B(s),
        deceleration_zone_A_B(s),
        v_constant(s),
        low_speed(s)
    )

#acceleration_zone_A_B(s)  # γ ≈ 1 Implies(acceleration_zone_A_B(s), Equals(s["gamma_A"], Real(1)))
#deceleration_zone_A_B(s)  # γ ≈ 0 Implies(deceleration_zone_A_B(s), Equals(s["gamma_A"], Real(0)))
#v_constant_A_B(s)         # γ ≈ constante
#low_speed_A_B(s)           # γ ≈ baixo


4. Atribuição de parâmetros por zona

In [4]:
def assign_parameters_A_B(s, acceleration_zone, deceleration_zone, v_constant_zone, low_speed_zone):
    return And(
        # gamma γ
        Implies(acceleration_zone(s), Equals(s["gamma_A"], Real(1))),
        Implies(deceleration_zone(s), Equals(s["gamma_A"], Real(0))),
        Implies(v_constant_zone(s), Equals(s["gamma_A"], Real(0))),
        Implies(low_speed_zone(s), Equals(s["gamma_A"], Real(0.1))),

        Equals(s["phi_A"], Real(0)),

        # V
        Implies(acceleration_zone(s), Equals(s["V_A"], Real(1.5))),
        Implies(deceleration_zone(s), Equals(s["V_A"], Real(1.0))),
        Implies(v_constant_zone(s), Equals(s["V_A"], Real(1.2))),
        Implies(low_speed_zone(s), Equals(s["V_A"], Real(0.5))),

        #  o coeficiente de atrito com a àgua, uma constante positiva que é independente do modo mas depende do navio
        GT(s["sigma_A"], Real(0))
    )


In [5]:
#---------------------------------------
#x_A = x0_A + z_A * cos(phi_A) 
#y_A = y0_A + z_A * sin(phi_A) 

def init_A_B(s):
    return And(
        Equals(s["sector_A"], Int(11)),   # setor inicial
        Equals(s["z_A"], Real(0)),        # posição relativa no setor
        Equals(s["v_A"], Real(0)),        # velocidade inicial
        Equals(s["tau_A"], Real(0)),      # tempo inicial
        Equals(s["x0_A"], Real(0)),       # ponto inicial do setor
        Equals(s["y0_A"], Real(1)),
        Equals(s["x_A"], Real(0)),        # posição atual x
        Equals(s["y_A"], Real(1))         # posição atual y
    )






5. Equações de fluxo do navio A

$$
\begin{cases}
\dot{v} + \sigma v = \gamma, & \text{se } v \le V \\
\dot{v} + \sigma v = \varepsilon, & \text{se } v > V \\
\dot{z} = v
\end{cases}
$$


In [6]:
# formulaas do enunciado do movimento linear
def vars_flow_A_B(s):
    return And(# v˙+σv=γ se v≤V => v˙=γ−σv
        
        Implies(LE(s["v_A"], s["V_A"]),Equals(s["v_A_dot"],Minus(s["gamma_A"], Times(s["sigma_A"], s["v_A"])))),

        #v˙+σv=ε se v>V => v˙=ε−σv
        Implies(GT(s["v_A"], s["V_A"]), Equals(s["v_A_dot"],Minus (s["epsilon_A"] , Times(s["sigma_A"],s["v_A"])))),

        #  # z˙ = v
        Equals(s["z_A_dot"], s["v_A"]),
        # τ˙ = 1

        Equals(s["tau_A_dot"], Real(1)),
        #ver se e aqui
        
        
# Se phi é constante por sector 
# Então cos(phi_A) = 1 e sin(phi_A) = 0
        
        Equals(s["x_A"], Plus(s["x0_A"], s["z_A"])),  # horizontal
        Equals(s["y_A"], s["y0_A"]),                  # vertical constante

        
        GE(Plus(s["x0_A"], s["z_A"]), Real(0)),
        LE(Plus(s["x0_A"], s["z_A"]), Real(1)),
        GE(s["y0_A"], Real(0)),
        LE(s["y0_A"], Real(1))
        
    )
    

6. Definir as variáveis do navio B

In [7]:
def VAR_B_A(i):
    s = {}
    # estado discreto
    s["sector_B"] = Symbol("sector_B" + str(i), INT)

    # variáveis contínuas de estado
    s["z_B"]   = Symbol("z_B"+ str(i), REAL)
    s["v_B"]   = Symbol("v_B"+ str(i), REAL)
    s["tau_B"] = Symbol("tau_B"+ str(i), REAL)

    #sendo $$(x_0,y_0)\,$$ o ponto do sector onde a trajetória se inicia, no inicio inicio é (0,1)
    s["x0_B"] = Symbol("x0_B"+ str(i), REAL)
    s["y0_B"] = Symbol("y0_B"+ str(i), REAL)
    s["x_B"] = Symbol("x_B", REAL)
    s["y_B"] = Symbol("y_B", REAL)

    # parâmetros associados ao modo (sector)
    # Os modos do navio  são definidos pelos sectores $$\,s\,$$. 
    # A cada modo estão associados  parâmetros racionais, constantes dentro de cada sector mas variando de sector para sector.

    s["gamma_B"]   = Symbol("gamma_B"+ str(i), REAL) #  aceleração linear  definida pela propulsão do navio
    s["epsilon_B"] = Symbol("epsilon_B"+ str(i), REAL) #é a  força exercida pela corrente no canal que actua a favor ou contra a trajectória
    s["phi_B"]     = Symbol("phi_B"+ str(i), REAL) #  é o rumo do navio definido como o ângulo que a trajetória do movimento linear faz com o eixo horizontal.
    s["V_B"]       = Symbol("V_B"+ str(i), REAL) # limite superior na velocidade do navio

    # o coeficiente de atrito com a àgua, uma constante positiva que é independente do modo mas depende do navio.
    s["sigma_B"] = Symbol("sigma_B"+ str(i), REAL)

    s["v_B_dot"] = Symbol("v_B_dot", REAL)
    s["z_B_dot"] = Symbol("z_B_dot", REAL)
    s["tau_B_dot"] = Symbol("tau_B_dot", REAL)
    return s

In [8]:
def init_B_A(s):
    return And(
        Equals(s["sector_B"], Int(12)),
        Equals(s["z_B"], Real(0)),
        Equals(s["v_B"], Real(0)),        # velocidade inicial
        Equals(s["V_B"], Real(1.5)),      # limite de velocidade
        Equals(s["gamma_B"], Real(1)),    # aceleração do setor
        Equals(s["sigma_B"], Real(0.1)),  # atrito
        Equals(s["tau_B"], Real(0)),
        Equals(s["x0_B"], Real(7)),
        Equals(s["y0_B"], Real(1)),
        Equals(s["x_B"], Real(7)),        # inicializa posição x
        Equals(s["y_B"], Real(1))         # inicializa posição y
    )


7. Definir as zonas 

Zona de aceleração : $$\,\{s_{1},s_{3}\}\,$$  $$\,\{s_{12},s_{14}\}\,$$ $$\,\gamma \simeq 0\,$$
Zona de desaceleração : $$\{s_{11},s_{13}\}\,$$ $$\{s_2,s_4\}\,$$  $$\,\gamma \simeq 1\,$$
Velocidade constante e de cruzeiro : $$\{s_5,s_7,s_9\}\,$$  $$\{s_6,s_8,s_{10}\}\,$$
Zona de velocidade baixa mais ou menos constante : $$\{s_0\}\,$$



In [9]:
# os modos de aceleracao e assim
def acceleration_zone_B_A(s):
    return Or(
        Equals(s["sector_B"], Int(1)),
        Equals(s["sector_B"], Int(3)),
        Equals(s["sector_B"], Int(12)),
        Equals(s["sector_B"], Int(14)),
    )

def deceleration_zone_B_A(s):
    return Or(
        Equals(s["sector_B"], Int(11)),
        Equals(s["sector_B"], Int(13)),
        Equals(s["sector_B"], Int(2)),
        Equals(s["sector_B"], Int(4)),
    )

def mode_B_A(s):
    return Or(
        acceleration_zone_B_A(s),
        deceleration_zone_B_A(s),
        v_constant(s),
        low_speed(s)
    )

8. Atribuição de parâmetros por zona

In [10]:
def assign_parameters_B_A(s, acceleration_zone, deceleration_zone, v_constant_zone, low_speed_zone):
    return And(
        # gamma γ
        Implies(acceleration_zone(s), Equals(s["gamma_B"], Real(1))),
        Implies(deceleration_zone(s), Equals(s["gamma_B"], Real(0))),
        Implies(v_constant_zone(s), Equals(s["gamma_B"], Real(0))),
        Implies(low_speed_zone(s), Equals(s["gamma_B"], Real(0.1))),

        Equals(s["phi_B"], Real(0)),

        # V
        Implies(acceleration_zone(s), Equals(s["V_B"], Real(1.5))),
        Implies(deceleration_zone(s), Equals(s["V_B"], Real(1.0))),
        Implies(v_constant_zone(s), Equals(s["V_B"], Real(1.2))),
        Implies(low_speed_zone(s), Equals(s["V_B"], Real(0.5))),

        # sigma > 0 
        GT(s["sigma_B"], Real(0))
    )


9. Equações de fluxo do navio A

$$
\begin{cases}
\dot{v} + \sigma v = \gamma, & \text{se } v \le V \\
\dot{v} + \sigma v = \varepsilon, & \text{se } v > V \\
\dot{z} = v
\end{cases}
$$


In [11]:
def vars_flow_B_A(s):
    return And(
        Implies(LE(s["v_B"], s["V_B"]), Equals(s["v_B_dot"], Minus(s["gamma_B"], Times(s["sigma_B"], s["v_B"])))),
        
        Implies(GT(s["v_B"], s["V_B"]),Equals(s["v_B_dot"], Minus(s["epsilon_B"], Times(s["sigma_B"], s["v_B"])))),
        
        Equals(s["z_B_dot"], s["v_B"]),

        Equals(s["tau_B_dot"], Real(1)),
#cos(ϕB​)=1, sin(ϕB​)=0
        Equals(s["x_B"], Plus(s["x0_B"], s["z_B"])),
        Equals(s["y_B"], s["y0_B"]),

        GE(Plus(s["x0_B"], s["z_B"]), Real(0)),
        LE(Plus(s["x0_B"], s["z_B"]), Real(1)),
        GE(s["y0_B"], Real(0)),
        LE(s["y0_B"], Real(1)),
        GE(s["v_B"], Real(0))
    )


10. Definir os modos do semáforo

In [12]:
def traffic(s_navios):
    s = {}
    s.update(s_navios)  # atualizar os setores
    s["green_A"] = Symbol("green_A", BOOL)
    s["green_B"] = Symbol("green_B", BOOL)
    s["yellow_A"] = Symbol("yellow_A", BOOL)
    s["yellow_B"] = Symbol("yellow_B", BOOL)
    s["t"] = Symbol("t", REAL)
    return s

11. Modelo do semáforo e sincronização das transições dos navios

- **Verde**   
  Permite que o navio transite para o próximo sector.

- **Amarelo**   
  Autoriza o navio a mover-se para o sector vizinho do sector onde está o outro navio.

- **Vermelho**  
  Representa não-sincronismo, ou seja, o navio não pode avançar.

In [13]:
# variavel continua unica nos semaforos
def traffic_flow(s):
    return Equals(s["t_dot"], Real(1))

def semaforo_update(s):
    """
    green = avança
    yellow = opcional
    red = não avança
    """

    green_A = Not(Equals(s["sector_A"], s["sector_B"]))
    green_B = Not(Equals(s["sector_B"], s["sector_A"]))

    yellow_A = And(LT(s["sector_A"], s["sector_B"]), Not(Equals(s["sector_A"] + 1, s["sector_B"])))
    yellow_B = And(GT(s["sector_B"], s["sector_A"]), Not(Equals(s["sector_B"] - 1, s["sector_A"])))

    return And(
        Iff(s["green_A"], green_A),
        Iff(s["green_B"], green_B),
        Iff(s["yellow_A"], yellow_A),
        Iff(s["yellow_B"], yellow_B))



def can_advance_A(s):
    return Or(s["green_A"], s["yellow_A"])


def can_advance_B(s):
    return Or(s["green_B"], s["yellow_B"])


12. Transição para o próximo sector do navio A

Esta função representa o **“jump”** do navio A para o próximo sector do canal.  
- **Guard**: verifica se o navio percorreu 1 km no sector atual e se o semáforo permite avançar.  
- **Reset**: reinicia as variáveis do sector (posição z, tempo τ, coordenadas iniciais x0 e y0) e atualiza o sector do navio.


In [14]:
def jump_next_sector_A(s_A, s_traffic, next_sector, x0_next, y0_next):
    guard = And(
        GE(s_A["z_A"], Real(1)),      # chegou ao fim do 1km
        can_advance_A(s_traffic)      #verifica se pode continuar p
    )

    reset = And(
        Equals(s_A["z_A"], Real(0)),
        Equals(s_A["tau_A"], Real(0)),
        Equals(s_A["x0_A"], Real(x0_next)),
        Equals(s_A["y0_A"], Real(y0_next)),
        Equals(s_A["sector_A"], Int(next_sector))
    )

    return And(guard, reset)


12. Transição para o próximo sector do navio B

Esta função representa o **“jump”** do navio B para o próximo sector do canal.  
- **Guard**: verifica se o navio percorreu 1 km no sector atual e se o semáforo permite avançar.  
- **Reset**: reinicia as variáveis do sector (posição z, tempo τ, coordenadas iniciais x0 e y0) e atualiza o sector do navio.

In [15]:
def jump_next_sector_B(s_B, s_traffic, next_sector, x0_next, y0_next):
    # guard depende do semáforo
    guard = And(
        GE(s_B["z_B"], Real(1)),
        can_advance_B(s_traffic)  # aqui s_traffic já tem green_B/yellow_B
    )

    reset = And(
        Equals(s_B["z_B"], Real(0)),
        Equals(s_B["tau_B"], Real(0)),
        Equals(s_B["x0_B"], Real(x0_next)),
        Equals(s_B["y0_B"], Real(y0_next)),
        Equals(s_B["sector_B"], Int(next_sector))
    )

    return And(guard, reset)


# Construir o FOTS que representa o sistema híbrido global: identificar o predicado que descreve a condição de segurança.

i. Funções auxiliares 

In [16]:
# Define todos os movimentos possíveis para o Navio A (Esquerda -> Direita)
def adjacente_A_B(atual, proximo):
    return Or(
        # Coluna 1 (Porto A) -> Coluna 2
        And(Equals(atual, Int(11)), Or(Equals(proximo, Int(5)), Equals(proximo, Int(7)))),
        And(Equals(atual, Int(13)), Or(Equals(proximo, Int(7)), Equals(proximo, Int(9)))),
        # Coluna 2 -> Coluna 3
        And(Or(Equals(atual, Int(5)), Equals(atual, Int(7))), Equals(proximo, Int(1))),
        And(Or(Equals(atual, Int(7)), Equals(atual, Int(9))), Equals(proximo, Int(3))),
        # Coluna 3 -> Centro (s0)
        And(Or(Equals(atual, Int(1)), Equals(atual, Int(3))), Equals(proximo, Int(0))),
        # Centro (s0) -> Coluna 4
        And(Equals(atual, Int(0)), Or(Equals(proximo, Int(2)), Equals(proximo, Int(4)))),
        # Coluna 4 -> Coluna 5
        And(Equals(atual, Int(2)), Or(Equals(proximo, Int(6)), Equals(proximo, Int(8)))),
        And(Equals(atual, Int(4)), Or(Equals(proximo, Int(8)), Equals(proximo, Int(10)))),
        # Coluna 5 -> Coluna 6 (Porto B)
        And(Or(Equals(atual, Int(6)), Equals(atual, Int(8))), Equals(proximo, Int(12))),
        And(Or(Equals(atual, Int(8)), Equals(atual, Int(10))), Equals(proximo, Int(14)))
    )

# Define todos os movimentos possíveis para o Navio B (Direita -> Esquerda)
def adjacente_B_A(atual, proximo):
    return Or(
        # Coluna 6 (Porto B) -> Coluna 5
        And(Equals(atual, Int(12)), Or(Equals(proximo, Int(6)), Equals(proximo, Int(8)))),
        And(Equals(atual, Int(14)), Or(Equals(proximo, Int(8)), Equals(proximo, Int(10)))),
        # Coluna 5 -> Coluna 4
        And(Or(Equals(atual, Int(6)), Equals(atual, Int(8))), Equals(proximo, Int(2))),
        And(Or(Equals(atual, Int(8)), Equals(atual, Int(10))), Equals(proximo, Int(4))),
        # Coluna 4 -> Centro (s0)
        And(Or(Equals(atual, Int(2)), Equals(atual, Int(4))), Equals(proximo, Int(0))),
        # Centro (s0) -> Coluna 3
        And(Equals(atual, Int(0)), Or(Equals(proximo, Int(1)), Equals(proximo, Int(3)))),
        # Coluna 3 -> Coluna 2
        And(Equals(atual, Int(1)), Or(Equals(proximo, Int(5)), Equals(proximo, Int(7)))),
        And(Equals(atual, Int(3)), Or(Equals(proximo, Int(7)), Equals(proximo, Int(9)))),
        # Coluna 2 -> Coluna 1 (Porto A)
        And(Or(Equals(atual, Int(5)), Equals(atual, Int(7))), Equals(proximo, Int(11))),
        And(Or(Equals(atual, Int(7)), Equals(atual, Int(9))), Equals(proximo, Int(13)))
    )

ii. Funções que definem o FOTS

In [17]:
def declare(i):
    # Criamos os estados individuais usando as funções da Alínea 1
    sA = VAR_A_B(i)
    sB = VAR_B_A(i)
    
    # Combinamos tudo no estado global 's'
    s = {**sA, **sB}
    
    s["x_A"] = Symbol("x_A" + str(i), REAL)
    s["y_A"] = Symbol("y_A" + str(i), REAL)
    s["x_B"] = Symbol("x_B" + str(i), REAL)
    s["y_B"] = Symbol("y_B" + str(i), REAL)
    
    # Variáveis de derivada (dots) para o fluxo
    s["v_A_dot"] = Symbol("v_A_dot" + str(i), REAL)
    s["z_A_dot"] = Symbol("z_A_dot" + str(i), REAL)
    s["tau_A_dot"] = Symbol("tau_A_dot" + str(i), REAL)
    
    s["v_B_dot"] = Symbol("v_B_dot" + str(i), REAL)
    s["z_B_dot"] = Symbol("z_B_dot" + str(i), REAL)
    s["tau_B_dot"] = Symbol("tau_B_dot" + str(i), REAL)

    s["t_dot"] = Symbol("t_dot" + str(i), REAL) # Requerido pelo traffic_flow
    
    # Adicionamos as variáveis do semáforo (Traffic Control)
    s = traffic(s) 
    return s

def init(s):
    # Combinamos as condições iniciais de A e B definidas na Alínea 1
    return And(
        init_A_B(s),
        init_B_A(s),
        Equals(s["t"], Real(0)),
        semaforo_update(s) # O semáforo começa calculado
    )

def trans(s, p):
    dt = p["t"] - s["t"]
    
    setor_A_simbolo = s["sector_A"]
    setor_B_simbolo = s["sector_B"]
    
    #Utilização das funções lambda para não dar erro do tipo unhashable type: 'dict'
    params_A = assign_parameters_A_B(s, 
                                     lambda _: acceleration_zone_A_B(s), 
                                     lambda _: deceleration_zone_A_B(s), 
                                     lambda _: v_constant(setor_A_simbolo), 
                                     lambda _: low_speed(setor_A_simbolo))
    
    params_B = assign_parameters_B_A(s, 
                                     lambda _: acceleration_zone_B_A(s), 
                                     lambda _: deceleration_zone_B_A(s), 
                                     lambda _: v_constant(setor_B_simbolo), 
                                     lambda _: low_speed(setor_B_simbolo))
    
    timed = And(
        dt > Real(0),
        params_A, params_B,
        vars_flow_A_B(s), vars_flow_B_A(s),
        Equals(p["v_A"], s["v_A"] + s["v_A_dot"] * dt),
        Equals(p["v_B"], s["v_B"] + s["v_B_dot"] * dt),
        Equals(p["z_A"], s["z_A"] + s["z_A_dot"] * dt),
        Equals(p["z_B"], s["z_B"] + s["z_B_dot"] * dt),
        LE(p["z_A"], Real(1.0)), LE(p["z_B"], Real(1.0)),
        Equals(p["sector_A"], s["sector_A"]), Equals(p["sector_B"], s["sector_B"]),
        semaforo_update(p)
    )

    jump_A = And(
        Equals(dt, Real(0)),
        s["z_A"] >= Real(1.0),            # Chegou ao fim do setor
        adjacente_A_B(s["sector_A"], p["sector_A"]), # Pode ir para QUALQUER adjacente
        Not(Equals(p["sector_A"], s["sector_B"])), 
        Equals(p["z_A"], Real(0)),
        Equals(p["v_A"], s["v_A"]),
        Equals(p["sector_B"], s["sector_B"]), # Navio B não se mexe
        Equals(p["z_B"], s["z_B"]),
        Equals(p["v_B"], s["v_B"]),
        semaforo_update(p)
    )

    jump_B = And(
        Equals(dt, Real(0)),
        s["z_B"] >= Real(1.0),
        adjacente_B_A(s["sector_B"], p["sector_B"]), # Pode ir para QUALQUER adjacente
        Not(Equals(p["sector_B"], s["sector_A"])),   
        Equals(p["z_B"], Real(0)),
        Equals(p["v_B"], s["v_B"]),
        Equals(p["sector_A"], s["sector_A"]), # Navio A não se mexe
        Equals(p["z_A"], s["z_A"]),
        Equals(p["v_A"], s["v_A"]),
        semaforo_update(p)
    )

    return Or(timed, jump_A, jump_B)

iii. Predicados de segurança

In [18]:
# Segurança suficiente: nunca os dois navios no mesmo setor.
def seguranca_suficiente(s):
    return Not(Equals(s["sector_A"], s["sector_B"]))

# Segurança forte: segurança suficiente + nenhum navio é forçado a imobilizar-se.
# Interpretado como: se um navio quer saltar (z >= 1), o semáforo deve estar verde.
def seguranca_forte(s):
    cond_A = Implies(GE(s["z_A"], Real(1.0)), s["green_A"])
    cond_B = Implies(GE(s["z_B"], Real(1.0)), s["green_B"])
    
    return And(seguranca_suficiente(s), cond_A, cond_B)

# Verificação, da segurança do sistema quer na versão suficiente como na versão forte. Utilizaremos BMC - Bounded Model Checking.

In [19]:
def bmc_always(declare,init,trans,prop,K):
    for k in range(1,K+1):
        with Solver(name="z3") as s:
            estados = [declare(i) for i in range(k+1)]

            s.add_assertion(init(estados[0]))

            for i in range(k):
                s.add_assertion(trans(estados[i], estados[i+1]))

            not_safe = Or(
                [Not(prop(estados[i])) for i in range(k+1)]
            )
            s.add_assertion(not_safe)

            if s.solve():
                print("Contra-exemplo encontrado")
                
                model = s.get_model()

                for i, estado in enumerate(estados):
                    print(f"Estado {i}:")
                    for nome, var in estado.items():
                        val = model.get_py_value(var)

                        if var.symbol_type().is_real_type():
                            val = float(val)
                            print(f"  {nome} = {val}")
                        else:
                            print(f"  {nome} = {val}")
                    print()
                return 
    print(f"Propriedade é válida até o bound {k}")

# Para verificar segurança suficiente
bmc_always(declare, init, trans, seguranca_suficiente, 20)

# Para verificar segurança forte
bmc_always(declare, init, trans, seguranca_forte, 20)    

Propriedade é válida até o bound 20
Propriedade é válida até o bound 20


Logo, para o bound de 20, o sistema mostra-se seguro tanto na versão suficiente quanto na versão forte